In [52]:
# testing uncentered-trained RNN 
# importing pytorch
import torch
import pandas as pd
from RNN_reconstructor import load_model_from_checkpoint


In [53]:
# running data config file
%run '/home/christianl/Zhang-Lab/Zhang Lab Code/Remote boilerplate/model_boilerplate_remote.py'


AttributeError: 'DataFrame' object has no attribute 'column'

In [54]:
# changing format of testing data to suit loading script 
x_test_df = pd.DataFrame(x_test, columns=input_labels)
y_test_df = pd.DataFrame(y_test, columns=output_labels)

NameError: name 'output_labels' is not defined

In [ ]:
# reconstructing trained RNN
model_dict = '/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/RNN/signaling_model.v1.pt'
net = pd.read_csv('/home/christianl/Zhang-Lab/Zhang Lab Data/Full data files/network(full).tsv', sep = '\t')

loaded_RNN = load_model_from_checkpoint(checkpoint_path= model_dict, node_names = input_labels, net= net, X_in= x_test_df, y_out= pd.DataFrame(y_test), device='cpu', use_exact_training_params=True)

LOADING WITH EXACT TRAINING PARAMETERS


/home/christianl/Zhang-Lab/Zhang Lab Code/Tuning/uncentered_RNN_tuning/RNN_reconstructor.py:402: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  



✓ Using EXACT parameters from benchmark.py

Configuration:
  seed: 888
  projection_amplitude_in: 1.2
  projection_amplitude_out: 1.2
  bionet_params: {'target_steps': 150, 'max_steps': 10, 'exp_factor': 50, 'tolerance': 1e-20, 'leak': 0.01}

Model dimensions:
  Total nodes: 16371
  Input TFs: 1197
  Output genes: 16100
  Edges: 1153904

INITIALIZING MODEL


KeyError: np.int64(0)

In [ ]:
# reconstructing trained RNN
model_dict = '/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/RNN/signaling_model.v1.pt'
net = pd.read_csv('/home/christianl/Zhang-Lab/Zhang Lab Data/Full data files/network(full).tsv', sep = '\t')

loaded_RNN = load_model_from_checkpoint(checkpoint_path= model_dict, node_names = input_labels, net= net, X_in= pd.DataFrame(x_test), y_out= pd.DataFrame(y_test), device='cpu', use_exact_training_params=True)

In [49]:
from RNN_reconstructor import load_model_from_checkpoint

model_dict = '/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/RNN/signaling_model.v1.pt'
net = pd.read_csv('/home/christianl/Zhang-Lab/Zhang Lab Data/Full data files/network(full).tsv', sep='\t')

# Works with numpy arrays!
loaded_RNN = load_model_from_checkpoint(
    model_dict,
    net=net,
    X_in=x_test,  # numpy array ✓
    y_out=y_test,  # numpy array ✓
    device='cpu'
)

LOADING WITH EXACT TRAINING PARAMETERS


/home/christianl/Zhang-Lab/Zhang Lab Code/Tuning/uncentered_RNN_tuning/RNN_reconstructor.py:402: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  



✓ Using EXACT parameters from benchmark.py

Configuration:
  seed: 888
  projection_amplitude_in: 1.2
  projection_amplitude_out: 1.2
  bionet_params: {'target_steps': 150, 'max_steps': 10, 'exp_factor': 50, 'tolerance': 1e-20, 'leak': 0.01}

Model dimensions:
  Total nodes: 16371
  Input TFs: 1197
  Output genes: 16100
  Edges: 1153904

⚠️  Using generic node names (node_0, node_1, ...)

INITIALIZING MODEL


AttributeError: 'numpy.ndarray' object has no attribute 'values'